<a href="https://colab.research.google.com/github/LucioCirelli/Test-Repo/blob/main/Modelo_Cartocor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  MODELO MATEMÁTICO DE OPTIMIZACIÓN GLOBAL DE PLANIFICACIÓN OPERATIVA: CARTOCOR

## Paquetes

In [1]:
"""
CARTOCOR: MODELO DE OPTIMIZACIÓN GLOBAL DE PLANIFICACIÓN OPERATIVA
"""
!pip install -q pyomo

from openpyxl import load_workbook

import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import *

import pandas as pd
model = ConcreteModel()

from google.colab import files
uploaded = files.upload()

archivo = 'Escenario_20240301_11_02.xlsx'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.3 MB/s eta 0:00:00


Saving Escenario_20240301_11_02.xlsx to Escenario_20240301_11_02.xlsx


## Parametros Usuario y Gams

In [ ]:
workbook = load_workbook(archivo)

# Situación Optima (0) o Actual (1)

sheet = workbook['Parametros_Usuario']
cell_value1 = sheet['B4'].value

run_sit_actual = cell_value

# Método Turnos "0-turnos libres, 1-turnos fijos por maq y mes + extras, 2-turnos fijos múltiplos de 20"

cell_value2 = sheet['B5'].value

turnos = cell_value2

# No correr ET especificas

cell_value3 = sheet['B6'].value

avoid_ET_especificas = cell_value3

# Min Max Cartón "0-no, 1-min-max de cartón a corrugar por fábrica"

cell_value4 = sheet['B7'].value

min_max_carton = cell_value4

# Min Max Abstecimiento "0-no, 1-por proveedor-mes, 2 por prov-papel-mes, 3-por proveedor-familia papel-mes, 4-prov-planta-mes "

cell_value5 = sheet['B8'].value

min_max_abast = cell_value5

# Min Max Viajes "0-no, 1-por CP, 2-por zona grande"

cell_value6 = sheet['B9'].value

min_max_viajes = cell_value6

# Límite de tiempo en segundos

sheet = workbook['Parametros_Gams']
cell_value7 = sheet['B4'].value

limite_tiempo = cell_value7

# Corte Relativo "% de Gap respecto de la mejor solución posible"

cell_value8 = sheet['B5'].value

corte_relativo = cell_value8

# Solver 0 - Gurobi ; 1 - Cplex

cell_value9 = sheet['B6'].value

solver = cell_value9

# Factor Hora Extra "Factor de consideración de las HE respecto de la Hora Normal"

sheet = workbook['Par_Operativos_Especiales']
cell_value10 = sheet['B4'].value

factor_HE = cell_value10

# Maxima Asignacion a Fabrica "Factor forzoso de máxima asignación a fábrica, 0-cant zonas de dist, 1-6 valor forzado"

cell_value11 = sheet['B5'].value

cant_PZ_user = cell_value11

# Max Máquinas x Fab "Máxima cantidad de máquinas que se pueden usar por fábrica para una ET,0-cant máq en fábrica, 1-6 user"

cell_value11 = sheet['B6'].value

maxF_M_user = cell_value11



1


## Sets

In [ ]:

########################## SETS ################################
#chequear si en vez de header conviene usar skiprows=range(0,1)

# mp = materia prima
MP1 = pd.read_excel(archivo,'Conjuntos',usecols='A',header=0)
MP2 = MP1.dropna()
MP3 = MP2.values.tolist()

MP = []
for i in MP3:
  for j in i:
    MP.append(j)

model.mp = Set(initialize=MP,doc='materia prima')

# p = ET

P1 = pd.read_excel(archivo,'Conjuntos',usecols='B',header=0)
P2 = P1.dropna()
P3 = P2.values.tolist()

P = []
for i in P3:
  for j in i:
    P.append(j)

model.p = Set(initialize=P,doc='ET')

# z = zona

Z1 = pd.read_excel(archivo,'Conjuntos',usecols='C',header=0)
Z2 = Z1.dropna()
Z3 = Z2.values.tolist()

Z = []
for i in Z3:
  for j in i:
    Z.append(j)

model.z = Set(initialize=Z,doc='Zona')

# t = meses

T1 = pd.read_excel(archivo,'Conjuntos',usecols='D',header=0)
T2 = T1.dropna()
T3 = T2.values.tolist()

T = []
for i in T3:
  for j in i:
    T.append(j)

model.t = Set(initialize=T,doc='meses')

# m = maquinas

M1 = pd.read_excel(archivo,'Conjuntos',usecols='E',header=0)
M2 = M1.dropna()
M3 = M2.values.tolist()

M = []
for i in M3:
  for j in i:
    M.append(j)

model.m = Set(initialize=M,doc='maquinas')

# f = fabrica

F1 = pd.read_excel(archivo,'Conjuntos',usecols='F',header=0)
F2 = F1.dropna()
F3 = F2.values.tolist()

F = []
for i in F3:
  for j in i:
    F.append(j)

model.f = Set(initialize=F,doc='fabricas')

# ff = fabrica copia --> ME AHORRO DECLARAR UN SET

model.ff = Set(initialize=F,doc='fabricas')

# pr = proveedores

PR1 = pd.read_excel(archivo,'Conjuntos',usecols='G',header=0)
PR2 = PR1.dropna()
PR3 = PR2.values.tolist()

PR = []
for i in PR3:
  for j in i:
    PR.append(j)

model.pr = Set(initialize=PR,doc='proveedores')

# c = tipo de carton

C1 = pd.read_excel(archivo,'Conjuntos',usecols='H',header=0)
C2 = C1.dropna()
C3 = C2.values.tolist()

C = []
for i in C3:
  for j in i:
    C.append(j)

model.c = Set(initialize=C,doc='tipo de carton')

# fmp = familia de materia prima

FMP1 = pd.read_excel(archivo,'Conjuntos',usecols='I',header=0)
FMP2 = FMP1.dropna()
FMP3 = FMP2.values.tolist()

FMP = []
for i in FMP3:
  for j in i:
    FMP.append(j)

model.fmp = Set(initialize=FMP,doc='familia de materia prima')

# tm = tipo de maquina

TM1 = pd.read_excel(archivo,'Conjuntos',usecols='J',header=0)
TM2 = TM1.dropna()
TM3 = TM2.values.tolist()

TM = []
for i in TM3:
  for j in i:
    TM.append(j)

model.tm = Set(initialize=TM,doc='tipo de maquina')

# pg = pegadora

PG1 = pd.read_excel(archivo,'ProductividadPegadora',usecols='A',header=0)
PG2 = PG1.dropna()
PG3 = PG2.values.tolist()

PG = []
for i in PG3:
  for j in i:
    PG.append(j)

model.pg = Set(initialize=PG,doc='pegadora')

# ab = abrochadora

AB1 = pd.read_excel(archivo,'ProductividadAbrochadora',usecols='A',header=0)
AB2 = AB1.dropna()
AB3 = AB2.values.tolist()

AB = []
for i in AB3:
  for j in i:
    AB.append(j)

model.ab = Set(initialize=AB,doc='abrochadora')



##################### SETS BIDIMENSIONALES ########################

# ET - Tipo de Maquina

ET_TM1 = pd.read_excel(archivo,'ConjuntosMultiples',usecols='AM:AN',header=0)
ET_TM2 = ET_TM1.dropna()
ET_TM3 = ET_TM2.values.tolist()

ET_TM3_tupla = ((p,tm) for [p,tm] in ET_TM3)

model.ET_TM = Set(initialize=ET_TM3_tupla, doc='et - tipo de maquina',dimen=2)

# Fabrica - Tipo de Maquina

F_TM1 = pd.read_excel(archivo,'ConjuntosMultiples',usecols='AP:AQ',header=0)
F_TM2 = F_TM1.dropna()
F_TM3 = F_TM2.values.tolist()

F_TM3_tupla = ((f,tm) for [f,tm] in F_TM3)

model.F_TM = Set(initialize=F_TM3_tupla, doc='fabrica - tipo de maquina',dimen=2)

# Fabrica - Tipo de Maquina - Maquina

F_TM_M1 = pd.read_excel(archivo,'ConjuntosMultiples',usecols='AS:AU',header=0)
F_TM_M2 = F_TM_M1.dropna()
F_TM_M3 = F_TM_M2.values.tolist()

F_TM_M3_tupla = ((f,tm,m) for [f,tm,m] in F_TM_M3)

model.F_TM_M = Set(initialize=F_TM_M3_tupla, doc='fabrica - tipo de maquina - nro maquina',dimen=3)

# Maquina - Tipo de maquina

M_TM1 = pd.read_excel(archivo,'ConjuntosMultiples',usecols='AW:AX',header=0)
M_TM2 = M_TM1.dropna()
M_TM3 = M_TM2.values.tolist()

M_TM3_tupla = ((m,tm) for [m,tm] in M_TM3)

model.M_TM = Set(initialize=M_TM3_tupla, doc='nro maquina - tipo de maquina',dimen=2)

# ET - nro de maquina

PM1 = pd.read_excel(archivo,'ConjuntosMultiples',usecols='A:B',header=0)
PM2 = PM1.dropna()
PM3 = PM2.values.tolist()

PM3_tupla = ((p,m) for [p,m] in PM3)

model.PM = Set(initialize=PM3_tupla, doc='et - nro maquina',dimen=2)

# nro de maquina - fabrica

MF1 = pd.read_excel(archivo,'ConjuntosMultiples',usecols='D:E',header=0)
MF2 = MF1.dropna()
MF3 = MF2.values.tolist()

MF3_tupla = ((m,f) for [m,f] in MF3)

model.MF = Set(initialize=MF3_tupla, doc='nro de maquina - fabrica',dimen=2)

# et - zona

PZ1 = pd.read_excel(archivo,'ConjuntosMultiples',usecols='G:H',header=0)
PZ2 = PZ1.dropna()
PZ3 = PZ2.values.tolist()

PZ3_tupla = ((p,z) for [p,z] in PZ3)

model.PZ = Set(initialize=PZ3_tupla, doc='et - zona',dimen=2)

# materia prima - et

MPP1 = pd.read_excel(archivo,'ConjuntosMultiples',usecols='J:K',header=0)
MPP2 = MPP1.dropna()
MPP3 = MPP2.values.tolist()

MPP3_tupla = ((mp,p) for [mp,p] in MPP3)

model.PZ = Set(initialize=MPP3_tupla, doc='materia prima - et',dimen=2)

# proveedor - materia prima - fabrica

PRMPF1 = pd.read_excel(archivo,'ConjuntosMultiples',usecols='M:O',header=0)
PRMPF2 = PRMPF1.dropna()
PRMPF3 = PRMPF2.values.tolist()

PRMPF3_tupla = ((pr,mp,f) for [pr,mp,f] in PRMPF3)

model.PRMPF = Set(initialize=PRMPF3_tupla, doc='proveedor - materia prima - fabrica',dimen=3)

# proveedor - materia prima

PRMP1 = pd.read_excel(archivo,'ConjuntosMultiples',usecols='Q:R',header=0)
PRMP2 = PRMP1.dropna()
PRMP3 = PRMP2.values.tolist()

PRMP3_tupla = ((pr,mp) for [pr,mp] in PRMP3)

model.PRMP = Set(initialize=PRMP3_tupla, doc='proveedor - materia prima',dimen=2)

# proveedor - fabrica

PRF1 = pd.read_excel(archivo,'ConjuntosMultiples',usecols='T:U',header=0)
PRF2 = PRF1.dropna()
PRF3 = PRF2.values.tolist()

PRF3_tupla = ((pr,f) for [pr,f] in PRF3)

model.PRF = Set(initialize=PRF3_tupla, doc='proveedor - fabrica',dimen=2)

# fabrica - zona

FZ1 = pd.read_excel(archivo,'ConjuntosMultiples',usecols='W:X',header=0)
FZ2 = FZ1.dropna()
FZ3 = FZ2.values.tolist()

FZ3_tupla = ((f,z) for [f,z] in FZ3)

model.FZ = Set(initialize=FZ3_tupla, doc='fabrica - zona',dimen=2)

# et - carton

PC1 = pd.read_excel(archivo,'ConjuntosMultiples',usecols='Z:AA',header=0)
PC2 = PC1.dropna()
PC3 = PC2.values.tolist()

PC3_tupla = ((p,c) for [p,c] in PC3)

model.PC = Set(initialize=PC3_tupla, doc='et - carton',dimen=2)

# materia prima - fabrica

MPF1 = pd.read_excel(archivo,'ConjuntosMultiples',usecols='AC:AD',header=0)
MPF2 = MPF1.dropna()
MPF3 = MPF2.values.tolist()

MPF3_tupla = ((mp,f) for [mp,f] in MPF3)

model.MPF = Set(initialize=MPF3_tupla, doc='materia prima - fabrica',dimen=2)

# materia prima - familia de materia prima - proveedor

MPFPR1 = pd.read_excel(archivo,'ConjuntosMultiples',usecols='AF:AH',header=0)
MPFPR2 = MPFPR1.dropna()
MPFPR3 = MPFPR2.values.tolist()

MPFPR3_tupla = ((mp,fmp,pr) for [mp,fmp,pr] in MPFPR3)

model.MPFPR = Set(initialize=MPFPR3_tupla, doc='materia prima - familia de materia prima - proveedor',dimen=3)

# asociación et - fabrica para corrugado

PF_C1 = pd.read_excel(archivo,'ETCorrugado',usecols='B:C',header=0)
PF_C2 = PF_C1.dropna()
PF_C3 = PF_C2.values.tolist()

PF_C3_tupla = ((p,f) for [p,f] in PF_C3)

model.PF_C = Set(initialize=PF_C3_tupla, doc='et - fabrica para corrugado',dimen=2)

# fabricas para pegadora

F_PG1 = pd.read_excel(archivo,'PegadoraFabrica',usecols='A',header=0)
F_PG2 = F_PG1.dropna()
F_PG3 = F_PG2.values.tolist()

F_PG4 = []
for i in F_PG3:
  for j in i:
    F_PG4.append(j)

model.F_PG = Set(initialize=F_PG4,doc='fabricas para pegadora')

# fabricas para abrochadora

F_AB1 = pd.read_excel(archivo,'AbrochadoraFabrica',usecols='A',header=0)
F_AB2 = F_AB1.dropna()
F_AB3 = F_AB2.values.tolist()

F_AB4 = []
for i in F_AB3:
  for j in i:
    F_AB4.append(j)

model.F_AB = Set(initialize=F_AB4,doc='fabricas para abrochadora')

# pegadora - fabrica

PGF1 = pd.read_excel(archivo,'ProductividadPegadora',usecols='A:B',header=0)
PGF2 = PGF1.dropna()
PGF3 = PGF2.values.tolist()

PGF3_tupla = ((pg,f) for [pg,f] in PGF3)

model.PGF = Set(initialize=PGF3_tupla, doc='pegadora - fabrica',dimen=2)

# abrochadora - fabrica

ABF1 = pd.read_excel(archivo,'ProductividadAbrochadora',usecols='A:B',header=0)
ABF2 = ABF1.dropna()
ABF3 = ABF2.values.tolist()

ABF3_tupla = ((ab,f) for [ab,f] in ABF3)

model.ABF = Set(initialize=ABF3_tupla, doc='abrochadora - fabrica',dimen=2)

# et - pegadora

PGP1 = pd.read_excel(archivo,'ETPegadora',usecols='A:B',header=0)
PGP2 = PGP1.dropna()
PGP3 = PGP2.values.tolist()

if len(PGP3) > 0:
  PGP3_tupla = ((p,pg) for [p,pg] in PGP3)
  model.PGP = Set(initialize=PGP3_tupla, doc='et - pegadora',dimen=2)

# et - abrochadora

ABP1 = pd.read_excel(archivo,'ETAbrochadora',usecols='A:B',header=0)
ABP2 = ABP1.dropna()
ABP3 = ABP2.values.tolist()

ABP3_tupla = ((p,ab) for [p,ab] in ABP3)

model.ABP = Set(initialize=ABP3_tupla, doc='et - abrochadora',dimen=2)

# et a no considerar
p_avoid1 = pd.read_excel(archivo,'ET_Avoid',usecols='A',header=0).dropna()
p_avoid2 = p_avoid1.values.tolist()

p_avoid3 = []
for i in p_avoid2:
  for j in i:
    p_avoid3.append(j)

model.p_avoid = Set(initialize=p_avoid3,doc='et a no considerar')

# Familia materia prima - proveedor

fmp_pr = ((fmp,pr) for fmp in model.fmp for pr in model.pr for mp in model.mp if (mp,fmp) in model.MPF and (mp,fmp,pr) in model.MPFPR)

model.FPR = Set(initialize = fmp_pr, doc = 'familia de mp - proveedor')

# Proveedor - Fabrica para Terminación

pr_f = ((p,f) for p in model.p for f in model.f for m in model.m if (p,m) in model.PM and (m,f) in model.MF)

model.PF_T = Set(initialize = pr_f, doc = 'proveedor - fabrica para terminación')

# Carton - Fabrica

c_f = ((c,f) for c in model.c for f in model.f for p in model.p if (p,c) in model.PC and (p,f) in model.PF_C)

model.CF = Set(initialize = c_f, doc = 'tipo  de carton - fabrica')

# ET - fabrica para abrochadora

ab_pf = ((p,f) for p in model.p for f in model.f if f in model.F_AB and p in model.ABP and (p,f) in model.PF_T)

model.ABPF = Set(initialize = ab_pf, doc = 'ET - fabrica para abrochado')

# ET - fabrica para pegadora

pg_pf = ((p,f) for p in model.p for f in model.f if p in model.PGP and f in model.F_PG and (p,f) in model.PF_T)

model.PGPF = Set(initialize = pg_pf, doc = 'ET - fabrica para pegadora')


########## PARAMETROS FABRICACION ####################

model.propiedades_fab = Set(initialize = ['scrap_op', 'scrap_ref', 'formato_prom', 'vel_prom', 'm2_hr', 'scrap_E+T','horas_turno', 'minC'], doc = 'set de param de fab')

######## ASIGNACIÓN DE SITUACIÓN ACTUAL ##############

if run_sit_actual == 1:
  AA1 = pd.read_excel(archivo,'AsignacionActual',usecols='A:B',header=0)
  AA2 = AA1.dropna()
  AA3 = AA2.values.tolist()

  AA3_tupla = ((p,f) for [p,f] in AA3)

  model.AA = Set(initialize=AA3_tupla, doc='asignación actual',dimen=2)





## Parámetros

In [ ]:
##################### PARAMETROS #########################

model.BigM = Param(initialize=99999)

# Parametros de Fabricacion

PARFAB1 = pd.read_excel(archivo,'ParametrosOperativos',usecols='A:I',skiprows=range(0,2))

aux = {(f,par_fab):PARFAB1[(PARFAB1['Fabrica'] == f)][par_fab] for f in model.f for par_fab in model.propiedades_fab}

# aca el aux deberia ir en el initialize del param, pero lo pusimos asi para chequear.

print(aux)

PARFAB = Param(model.f, model.propiedades_fab, initialize = aux)

print(PARFAB1)



#for key in PARFAB:
 #    print(PARFAB[key].head())

#print(type(PARFAB))




## Variables Binarias

In [ ]:
###### Variables Binarias #########

model.X_TRANSF = Var(model.f, model.ff, within = Binary, doc = 'si se transfiere semi-elaborado de f a ff')

model.X_M = Var(model.p, model.m, within = Binary, doc = 'Asignacion del producto p en la maquina m en el periodo analizado')

model.X_F_C = Var(model.p, model.f, within = Binary, doc = 'Asignacion del producto p en la fabrica f en el periodo analizado para corrugar')

model.X_F_T = Var(model.p, model.f, within = Binary, doc = 'Asignacion del producto p en la fabrica f en el periodo analizado para terminar')

model.X_M_AB = Var(model.p, model.ab, within = Binary, doc = 'Asignacion del producto p en la abrochadora ab en el periodo analizado')

model.X_M_PG = Var(model.p, model.pg, within = Binary, doc = 'Asignacion del producto p en la pegadora pg en el periodo analizado')

model.X_C = Var(model.c, model.f, model.t, within = Binary, doc = 'Decision de fabricar el carton c en la fabrica f en el periodo t')

model.X_FZ = Var(model.f, model.zg, model.t, within = Binary, doc = 'Decision de abastecer a la zona z desde la fabrica f en el periodo t')

model.X_M_adic = Var(model.m, within = Binary, doc = 'decision de agregar turnos adicionales en la maquina m')

model.X_C_adic = Var(model.f, within = Binary, doc = 'decision de agregar turnos adicionales en la corrugadora de la fabrica f')

model.X_A_PR = Var(model.pr, model.t, within = Binary, doc = 'Decision de comprar al proveedor pr en el periodo t')

model.X_A_PRMP = Var(model.pr, model.mp, model.t, within = Binary, doc = 'Decision de comprar materia prima mp al proveedor pr en el periodo t')

model.X_A_FAM = Var(model.pr, model.fmp, model.t, within = Binary, doc = 'Decision de comprar de la familia de materia prima mp al proveedor pr en el periodo t')

model.X_A_F = Var(model.pr, model.f, model.t, within = Binary, doc = 'Decision de comprar a un proveedor y abastecer a planta f en el periodo t')



## Variables Enteras

In [ ]:
model.Y_M_turnos = Var(model.m, model.t, within = PositiveIntegers)

model.Y_M_turnos_extras = Var(model.m, model.t, within = PositiveIntegers)

model.Y_C_turnos = Var(model.f, model.t, within = PositiveIntegers)

model.Y_C_turnos_extras = Var(model.f, model.t, within = PositiveIntegers)

model.Y_AB_turnos = Var(model.ab, model.t, within = PositiveIntegers)

model.Y_AB_turnos_extras = Var(model.ab, model.t, within = PositiveIntegers)

model.Y_PG_turnos = Var(model.pg, model.t, within = PositiveIntegers)

model.Y_PG_turnos_extras = Var(model.pg, model.t, within = PositiveIntegers)

model.Y_M_turnos_f = Var(model.m, within = PositiveIntegers)

model.Y_C_turnos_f = Var(model.f, within = PositiveIntegers)

model.NB = Var(model.f, model.tm, model.t, within = PositiveIntegers, doc = 'numero de bloques de dias habiles que representan el conjunto turno+operario por fabrica y tipo de maquina')

model.NB_C = Var(model.f, model.t, within = PositiveIntegers, doc = 'numero de bloques de dias habiles que representan el conjunto turno+operario por el uso de la corrugadora en una fabrica f')


## Variables Positivas

In [ ]:
# Corrugado y transferencia

model.Q_C = Var(model.f, model.p, model.t, within = NonNegativeReals, doc = 'corrugado [un] de la ET p en la fábrica f en el período t')

model.Q_TRANSF = Var(model.p, model.f, model.ff, model.t, within = NonNegativeReals, doc = 'transferencia [un] de la ET p de la fábrica f a la fábrica f')

# Produccion y variables operativas

model.Q_P_M = Var(model.m, model.p, model.t, within = NonNegativeReals, doc = 'Cantidad de cada producto p a ser fabricado en cada maquina m en cada periodo t')

model.Q_P_AB = Var(model.ab, model.p, model.t, within = NonNegativeReals, doc = 'Cantidad de cada producto p a ser fabricado en cada maquina de abrochado ab en cada periodo t')

model.Q_P_PG = Var(model.pg, model.p, model.t, within = NonNegativeReals, doc = 'Cantidad de cada producto p a ser fabricado en cada maquina de pegadora pg en cada periodo t')



model.Q_A = Var(model.mp, model.pr, model.f, model.t, within = NonNegativeReals, doc = 'Cantidad de materia prima mp enviada a la fabrica desde el proveedor pr durante el periodo t')

model.Q_E = Var(model.p, model.f, model.z, model.t, within = NonNegativeReals, doc = 'Cantidad de producto p entregada desde la fabrica f a la zona z durante el periodo t')

model.REQ_MP = Var(model.mp, model.f, model.t, within = NonNegativeReals, doc = 'Requerimientos totales de cada materia prima en cada fabrica en cada periodo')



model.CANT_PR = Var(model.pr, model.t, within = NonNegativeReals)

model.CANT_PRMP = Var(model.pr, model.mp, model.t, within = NonNegativeReals)

model.CANT_PRFAM = Var(model.pr, model.fmp, model.t, within = NonNegativeReals)

model.CANT_PRFAB = Var(model.pr, model.f, model.t, within = NonNegativeReals)




model.CANT_FZ = Var(model.f, model.zg, model.t, within = NonNegativeReals, doc = 'cantidad de viajes realizados desde la fabrica f a la zona z en el periodo t')

model.CANT_CORR_C = Var(model.c, model.f, model.t, within = NonNegativeReals, doc = 'cantidad de carton c en [m2] fabricado en la fabrica f en el periodo t')



model.DISP_HS_M = Var(model.m, model.t, within = NonNegativeReals, doc = 'disponibilidad de horas normales en maquinas')

model.DISP_HS_EXTRA_M = Var(model.m, model.t, within = NonNegativeReals, doc = 'disponibilidad de horas extra en maquinas')

model.DISP_HS_C = Var(model.f, model.t, within = NonNegativeReals, doc = 'disponibilidad de horas normales en corrugadora de la fabrica f')

model.DISP_HS_EXTRA_C = Var(model.f, model.t, within = NonNegativeReals, doc = 'disponibilidad de horas extra en corrugadora de la fabrica f')



model.DISP_HS_AB = Var(model.ab, model.t, within = NonNegativeReals, doc = 'disponibilidad de horas normales')

model.DISP_HS_EXTRA_AB = Var(model.ab, model.t, within = NonNegativeReals, doc = 'disponibilidad de horas extras')

model.DISP_HS_FASON_AB = Var(model.ab, model.t, within = NonNegativeReals, doc = 'tiempo tercerizado para la maquina m en el tiempo t')



model.DISP_HS_PG = Var(model.pg, model.t, within = NonNegativeReals,doc = 'disponibilidad de horas normales')

model.DISP_HS_EXTRA_PG = Var(model.pg, model.t, within = NonNegativeReals, doc = 'disponibilidad de horas extras')

model.DISP_HS_FASON_PG = Var(model.pg, model.t, within = NonNegativeReals, doc = 'tiempo tercerizado para la maquina m en el tiempo t')



model.TIEMPO_M = Var(model.m, model.t, within = NonNegativeReals, doc = 'tiempo total de uso de máquina m por período')

model.TIEMPO_C = Var(model.f, model.t, within = NonNegativeReals, doc = 'tiempo total de uso de la corrugadora de la fabrica f por período')

model.TIEMPO_AB = Var(model.ab, model.f, model.t, within = NonNegativeReals, doc = 'tiempo necesario de abrochado en la fabrica f en el periodo t')

model.TIEMPO_PG = Var(model.pg, model.f, model.t, within = NonNegativeReals, doc = 'tiempo necesario de pegado en la fabrica f en el periodo t')



model.CT_Abast = Var(within = NonNegativeReals, doc = 'Costo total de abastecimiento')

model.CT_Ent = Var(within = NonNegativeReals, doc = 'Costo total de entrega o distribucion')

model.CT_Prod_T = Var(within = NonNegativeReals, doc = 'costo productivo asociado al uso de la maquina')

model.CT_Prod_C = Var(within = NonNegativeReals, doc = 'costo productivo asociado al uso de la corrugadora')

model.CT_FASON_M= Var(within = NonNegativeReals,  doc = 'costo productivo asociado a la tercerizacion de la maquina')

model.CT_FASON_C = Var(within = NonNegativeReals, doc = 'costo productivo asociado a la tercerizacion de la corrugadora')

model.CT_FASON_PG = Var(within = NonNegativeReals,  doc = 'costo productivo asociado a la tercerizacion de la pegadora')

model.CT_FASON_AB = Var(within = NonNegativeReals,  doc = 'costo productivo asociado a la tercerizacion de la abrochadora')

model.CT_TRANSF = Var(within = NonNegativeReals)



model.REMANENTE_NB = Var(model.f, model.tm, model.t, within = NonNegativeReals, doc = 'remanente del numero de bloques asignado')

model.REMANENTE_NB_C = Var(model.f, model.t, within = NonNegativeReals, doc = 'remanente del numero de bloques asignado')



model.Req_C_hr = Var(model.f, model.t, within = NonNegativeReals, doc = 'Req total en horas')

model.Req_C_m2 = Var(model.f, model.t, within = NonNegativeReals, doc = 'Requerimiento total mensual de corrugado en m2 por fábrica')

model.Q_C_Input_Total = Var(model.f, model.t, within = NonNegativeReals, doc = 'total de m2 a ingresar a corrugado')




## Variables Slack

In [ ]:
##### SLACK INTEGER #####

model.Y_M_turnos_slack = Var(model.m, model.t, within = PositiveIntegers)

model.Y_C_turnos_slack = Var(model.f, model.t, within = PositiveIntegers)

###### SLACK POSITIVE ######

model.slack_fab = Var(model.f, model.t, within = NonNegativeReals)

model.slack_Transf = Var(model.f, model.p, model.t, within = NonNegativeReals)

model.slack_C = Var(model.f, model.p, model.t, within = NonNegativeReals)

model.Slack_HS_M = Var(model.m, model.t, within = NonNegativeReals)

model.SLACK_HS_c = Var(model.f, model.t, within = NonNegativeReals)

model.slack_ent = Var(model.p, model.z, model.t, within = NonNegativeReals)

model.slack_minC = Var(model.c, model.f, model.t, within = NonNegativeReals)

model.slack_maxC = Var(model.c, model.f, model.t, within = NonNegativeReals)





## Ecuaciones Main

In [ ]:
###### ASIGNACIONES ###################################################################################################################

# R1 - Asignación de producto a maquina

def R1_rule(model, p, f):
    if (p, f) in model.PF_T:
      return sum(model.X_M[p, m] for m in model.m if (p, m) in model.PM and (m, f) in model.MF) <= model.MaxF_M[f] * model.X_F_T[p, f]

model.R1 = Constraint(model.p, model.f, rule=R1_rule, doc = 'Asignación de producto a maquina')

# R2a - Asignacion de producto a fabrica para TERMINACIÓN

def R2a_rule(model,p):
  return sum(model.X_F_T[p,f] for f in model.f if (p,f) in model.PF_T) <= model.Cant_PZ[p]

model.R2a = Constraint(model.p, rule = R2a_rule, doc = 'Asignacion de producto a fabrica para TERMINACIÓN')

# R2b - Asignacion de producto a fabrica para CORRUGADO

def R2b_rule(model,p):
  return sum(model.X_F_C[p,f] for f in model.f if (p,f) in model.PF_C) <= model.Cant_PZ[p]

model.R2b = Constraint(model.p, rule = R2b_rule, doc = 'Asignacion de producto a fabrica para CORRUGADO')

# R3a - Asignación de las ET a fábrica para CORRUGADO

def R3a_rule(model, f, p, t):
  if (p,f) in model.PF_C and (p,t) in model.PT:
    return model.Q_C[f,p,t] <= model.X_F_C[p,f]*model.BigM

model.R3a = Constraint(model.f, model.p, model.t, rule = R3a_rule, doc = 'Asignación de las ET a fábrica para CORRUGADO')

###### BALANCES DE MASA #########################################################################################################

# R3b - Transferencia entre fábricas

def R3b_rule(model, f, p, t):
  if (p,f) in model.PF_C and (p,t) in model.PT:
    return model.Q_C[f,p,t] + model.slack_C[f,p,t] == sum(model.Q_TRANSF[p,f,ff,t] for ff in model.ff if (f,ff) in model.FFF )

model.R3b = Constraint(model.f, model.p, model.t, rule = R3b_rule, doc = 'Transferencia entre fábricas')

# R3c - Maxima capacidad de corrugado en M2

def R3c_rule(model, f, t):
  return sum(1000*model.Q_C[f,p,t]*model.m2_u[p] for p in model.p if (p,f) in model.PF_C and (p,t) in model.PT) / (1 - model.par_fab[f,'scrap_op']) / (1 - model.par_fab[f,'scrap_ref']) == model.Req_C_m2[f,t]

model.R3c = Constraint(model.f, model.t, rule = R3c_rule, doc = 'Maxima capacidad de corrugado en M2')

# R3c2 - Maxima capacidad de corrugado en M2

def R3c2_rule(model, f, t):
  return model.Req_C_m2[f,t] / model.par_fab[f,'m2_hr'] == model.Req_C_hr[f,t]

model.R3c2 = Constraint(model.f, model.t, rule = R3c2_rule, doc = 'Maxima capacidad de corrugado en M2')

# R3f - alimentación a corrugado total

def R3f_rule(model, f, t):
  return model.Q_C_Input_Total[f,t] == model.Req_C_m2[f,t] / (1 - model.par_fab[f, 'scrap_E+T'])

model.R3f = Constraint(model.f, model.t, rule = R3f_rule, doc = 'alimentación a corrugado total')

# R3c_min - Limite minimo transferencia y vinculacion con binaria

def R3c_min_rule(model, f, ff, t):
  if (f,ff) in model.FFF:
    return sum(model.Q_TRANSF[p,f,ff,t]*1000 for p in model.p if (p,f) in model.PF_C and (p,t) in model.PT and (p,ff) in model.PF_T) >= model.X_TRANSF[f,ff]*model.min_transf[f,ff]

model.R3c_min = Constraint(model.f, model.ff, model.t, rule = R3c_min_rule, doc = 'Limite minimo transferencia y vinculacion con binaria')

# R3c_max - Limite maximo transferencia y vinculación con binaria

def R3c_max_rule(model, f, ff, t):
  if (f,ff) in model.FFF:
    return sum(model.Q_TRANSF[p,f,ff,t]*1000 for p in model.p if (p,f) in model.PF_C and (p,t) in model.PT and (p,ff) in model.PF_T) <= model.X_TRANSF[f,ff]*model.max_transf[f,ff]

model.R3c_max = Constraint(model.f, model.ff, model.t, rule = R3c_max_rule, doc = 'Limite maximo transferencia y vinculación con binaria')

# R3d - Asignacion de transferencias a maquinas

def R3d_rule(model, f, p, t):
  if (p,f) in model.PF_T and (p,t) in model.PT:
    return sum(model.Q_TRANSF[p,ff,f,t] for ff in model.ff if (ff,f) in model.FFF and (p,ff) in model.PF_C) + model.slack_Transf[f,p,t] == sum(model.Q_P_M[m,p,t] for m in model.m if (m,f) in model.MF and (p,m) in model.PM)

model.R3d = Constraint(model.f, model.p, model.t, rule = R3d_rule, doc = 'Asignacion de transferencias a maquinas')

# R3 - Vinculacion de la variable binaria con la variable positiva

def R3_rule(model, p, m, t):
    if (p, m) in model.PM and (p, t) in model.PT:
        return model.Q_P_M[m, p, t] <= model.demT[p, t] / 1000 * model.X_M[p, m]

model.R3 = Constraint(model.p, model.m, model.t, rule = R3_rule, doc = 'Vinculacion de la variable binaria con la variable positiva')

def Q_P_M_fix_rule1(model, m, p, t):
    if not (p, t) in model.PT:
        return model.Q_P_M[m, p, t] == 0

model.Q_P_M_fix1 = Constraint(model.m, model.p, model.t, rule=Q_P_M_fix_rule1)

def Q_P_M_fix_rule2(model, m, p, t):
    if not (p, m) in model.PM:
        return model.Q_P_M[m, p, t] == 0

model.Q_P_M_fix2 = Constraint(model.m, model.p, model.t, rule=Q_P_M_fix_rule2)

# R11a - Calculo de la cantidad a fabricar de carton c en la fabrica f en el periodo t

def R11a_rule(model, c, f, t):
  if (c,f) in model.CF:
    return model.CANT_CORR_C[c,f,t] == sum(model.Q_C[f,p,t]*1000*model.m2_u[p] for p in model.p if (p,f) in model.PF_C and (p,t) in model.PT and (p,c) in model.PC) / (1 - model.par_fab[f,'scrap_op']) / (1 - model.par_fab[f,'scrap_ref'])

model.R11a = Constraint(model.c, model.f, model.t, rule = R11a_rule, doc = 'Calculo de la cantidad a fabricar de carton c en la fabrica f en el periodo t')

# R4 - Requerimientos totales de cada materia prima en cada fabrica y periodo considerando scraps

def R4_rule(model,mp, f, t):
  return model.REQ_MP[mp,f,t] == sum(model.Q_C[f,p,t]*1000/model.demT[p,t]*model.reqmp[p,t,mp] for p in model.p if (p,mp) in model.MPP and (p,f) in model.PF_C and (p,t) in model.PT) / (1 - model.par_fab[f,'scrap_op']) / ( 1 - model.par_fab[f,'scrap_ref']) / ( 1 - model.par_fab[f,'scrap_E+T'])

model.R4 = Constraint(model.mp, model.f, model.t, rule = R4_rule, doc = 'Requerimientos totales de cada materia prima en cada fabrica y periodo considerando scraps')

# R5 - Abastecimiento de las materias primas totales en cada fabrica en cada periodo

def R5_rule(model,mp, f, t):
  return sum(model.Q_A[mp,pr,f,t] for pr in model.pr if (pr,mp,f) in model.PRMPF and (pr,f) in model.PRF) == model.REQ_MP[mp,f,t]

model.R5 = Constraint(model.mp, model.f, model.t, rule = R5_rule, doc = 'Abastecimiento de las materias primas totales en cada fabrica en cada periodo')

# R7 - Balance de flujo de salida

def R7_rule(model, p, f, t):
  if (p,f) in model.PF_T and (p,t) in model.PT:
    return sum(model.Q_P_M[m,p,t] for m in model.m if (p,m) in model.PM and (m,f) in model.MF) == sum(model.Q_E[p,f,z,t] for z in model.z if (f,z) in model.FZ)

model.R7 = Constraint(model.p, model.f, model.t, rule = R7_rule, doc = 'Balance de flujo de salida')

# R8 - Volumenes de entrega y distribucion

def R8_rule(model, p, z, t):
  if (p,z) in model.PZ and (p,t) in model.PT:
    return sum(model.Q_E[p,f,z,t] for f in model.f if (f,z) in model.FZ and (p,f) in model.PF_T) + model.slack_ent[p,z,t] == model.dem[p,z,t] / 1000

model.R8 = Constraint(model.p, model.z, model.t, rule = R8_rule, doc = 'Volumenes de entrega y distribucion')

####### TIEMPOS y CAPACIDAD TOTALES MÁQUINAS TERMINADORAS Y CORRUGADORA #######################################################################################

## CAPACIDAD DE TERMINACION

# R6a - Cumplimiento disponibilidad mensual en horas para cada maquina m

def R6a_rule(model, m, t):
  if not (m, 'PLANCHAS') in model.M_TM:
    return model.TIEMPO_M[m,t] <= model.DISP_HS_M[m,t] + model.DISP_HS_EXTRA_M[m,t] + model.Slack_HS_M[m,t]

model.R6a = Constraint(model.m, model.t, rule = R6a_rule, doc = 'Cumplimiento disponibilidad mensual en horas para cada maquina m')

# R6b - Calculo de tiempo por máquina m

def R6b_rule(model, f, m, t):
  if (m,f) in model.MF and not (m,'PLANCHAS') in model.M_TM:
    return sum(model.Q_P_M[m,p,t]*model.m2_u[p]*1000/model.vel_m[p,m] for p in model.p if (p,m) in model.PM and (p,t) in model.PT) / (1 - model.par_fab[f,'scrap_op']) == model.TIEMPO_M[m,t]

model.R6b = Constraint(model.f, model.m, model.t, rule = R6b_rule, doc = 'Calculo de tiempo por máquina m')

## CAPACIDAD DE CORRUGADO

# R6c - Cumplimiento disponibilidad mensual en horas para cada corrugadora de la fabrica f

def R6c_rule(model, f, t):
  return model.TIEMPO_C[f,t] <= model.DISP_HS_C[f,t] + model.DISP_HS_EXTRA_C[f,t] + model.Slack_HS_C[f,t]

model.R6c = Constraint(model.f, model.t, rule = R6c_rule, doc = 'Cumplimiento disponibilidad mensual en horas para cada corrugadora de la fabrica f')

# R6d - Calculo de tiempo por corrugadora

def R6d_rule(model, f, t):
  return sum(model.Q_C[f,p,t]*model.m2_u[p]*1000/model.vel_c[f] for p in model.p if (p,f) in model.PF_C and (p,t) in model.PT) / (1 - model.par_fab[f,'scrap_op']) / ( 1 - model.par_fab[f,'scrap_ref']) == model.TIEMPO_C[f,t]

model.R6d = Constraint(model.f, model.t, rule = R6d_rule, doc = 'Calculo de tiempo por corrugadora')

# R6e - minimo tiempo a cumplir de corrugado

def R6e_rule(model, f, t):
  return model.TIEMPO_C[f,t] >= model.DISP_HS_C[f,t]*model.min_corr[f]/100

model.R6e = Constraint(model.f, model.t, rule = R6e_rule, doc = 'R6e - minimo tiempo a cumplir de corrugado')


## Ecuaciones Turnos

In [ ]:
############# turnos == 0 ---> TURNOS LIBRES POR MAQUINA Y MES ###################################################

if turnos == 0:

  # R_min_Corr

  def R_min_Corr_rule(model, f, t):
    return model.Y_C_turnos[f,t] >= model.turnos_c[f,t]*model.min_corr[f]/100

  model.R_min_Corr = Constraint(model.f, model.t, rule = R_min_Corr_rule, doc = '')

  # RTurnos1 - Maximo turnos disponibles maquinas

  def RTurnos1_rule(model, m, t):
    return model.Y_M_turnos[m,t] <= model.turnos_m[m,t]

  model.RTurnos1 = Constraint(model.m, model.t, rule = RTurnos1_rule, doc = 'Maximo turnos disponibles maquinas')

  # RTurnos2 - Maximo turnos extras disponibles maquinas

  def RTurnos2_rule(model, m, t):
    return model.Y_M_turnos_extras[m,t] <= model.turnos_extra_m[m,t]

  model.RTurnos2 = Constraint(model.m, model.t, rule = RTurnos2_rule, doc = 'Maximo turnos extras disponibles maquinas')

  # RTurnos3 - Disponibilidad horas normales maquinas

  def RTurnos3_rule(model, m, t):
    return model.DISP_HS_M[m,t] == model.Y_M_turnos[m,t]*model.horas_turno_fab

  model.RTurnos3 = Constraint(model.m, model.t, rule = RTurnos3_rule, doc = 'Disponibilidad horas normales maquinas')

  # RTurnos4 - Disponibilidad horas extras maquinas

  def RTurnos4_rule(model, m, t):
    return model.DISP_HS_EXTRA_M[m,t] == model.Y_M_turnos_extras[m,t]*model.horas_turno_fab

  model.RTurnos4 = Constraint(model.m, model.t, rule = RTurnos4_rule, doc = 'Disponibilidad horas extras maquinas')

  # RTurnos5 - Disponibilidad horas fason maquinas

  def RTurnos5_rule(model, m, t):
    return model.SLACK_HS_M[m,t] == model.Y_M_turnos_slack[m,t]*model.horas_turno_fab

  model.RTurnos5 = Constraint(model.m, model.t, rule = RTurnos5_rule, doc = 'Disponibilidad horas fason maquinas')

  # RTurnos1_c - Maximo turnos disponibles corrugadoras

  def RTurnos1_C_rule(model, m, t):
    return model.Y_C_turnos[m,t] <= model.turnos_c[m,t]

  model.RTurnos1_C = Constraint(model.m, model.t, rule = RTurnos1_C_rule, doc = 'Maximo turnos disponibles corrugadoras')

  # RTurnos2_C - Maximo turnos extras disponibles corrugadoras

  def RTurnos2_C_rule(model, m, t):
    return model.Y_C_turnos_extras[m,t] <= model.turnos_extra_c[m,t]

  model.RTurnos2_C = Constraint(model.m, model.t, rule = RTurnos2_C_rule, doc = 'Maximo turnos extras disponibles corrugadoras')

  # RTurnos3_C - Disponibilidad horas normales corrugadoras

  def RTurnos3_C_rule(model, m, t):
    return model.DISP_HS_C[m,t] == model.Y_C_turnos[m,t]*model.horas_turno_fab

  model.RTurnos3 = Constraint(model.m, model.t, rule = RTurnos3_C_rule, doc = 'Disponibilidad horas normales corrugadoras')

  # RTurnos4_C - Disponibilidad horas extras corrugadoras

  def RTurnos4_C_rule(model, m, t):
    return model.DISP_HS_EXTRA_C[m,t] == model.Y_C_turnos_extras[m,t]*model.horas_turno_fab

  model.RTurnos4_C = Constraint(model.m, model.t, rule = RTurnos4_C_rule, doc = 'Disponibilidad horas extras corrugadoras')

  # RTurnos5_C - Disponibilidad horas fason corrugadoras

  def RTurnos5_C_rule(model, m, t):
    return model.SLACK_HS_C[m,t] == model.Y_C_turnos_slack[m,t]*model.horas_turno_fab

  model.RTurnos5_C = Constraint(model.m, model.t, rule = RTurnos5_C_rule, doc = 'Disponibilidad horas fason corrugadoras')

################## turnos == 1 --> TURNOS FIJOS PARA TODO EL HORIZONTE  ##########################################################################

if turnos == 1:

  model.M_turnos = Param(initialize = 200)

  # R17a - Calculo de la disponibilidad de horas por maquina y por mes

  def R17a_rule(model, m, t):
    return model.DISP_HS_M[m,t] == model.Y_M_turnos_f[m]*model.horas_turno_fab

  model.R17a = Constraint(model.m, model.t, rule = R17a_rule, doc = 'Calculo de la disponibilidad de horas por maquina y por mes')

  # R17b - Calculo de la disponibilidad de horas extras por maquina y por mes

  def R17b_rule(model, m, t):
    return model.DISP_HS_EXTRA_M[m,t] == model.Y_M_turnos_extras[m]*model.horas_turno_fab

  model.R17b = Constraint(model.m, model.t, rule = R17b_rule, doc = 'Calculo de la disponibilidad de horas extras por maquina y por mes')

  # R17c - Cota maxima de turnos fijos y asociacion con la binaria

  def R17c_rule(model, m):
    return model.Y_M_turnos_f[m] <= model.turnos_fijos_min_m[m] - 1 + model.X_M_adic[m]

  model.R17c = Constraint(model.m, rule = R17c_rule, doc = 'Cota maxima de turnos fijos y asociacion con la binaria')

  # R17d - Cota minima de turnos fijos y asociacion con la binaria

  def R17d_rule(model, m):
    return model.turnos_fijos_min_m[m] - model.Y_M_turnos_f[m] <= model.M_turnos * ( 1 - model.X_M_adic[m] )

  model.R17d = Constraint(model.m, rule = R17d_rule, doc = 'Cota minima de turnos fijos y asociacion con la binaria')

  # R17e - Eleccion de usar turnos adicionales

  def R17e_rule(model, m, t):
    return model.Y_M_turnos_extras[m,t] <= model.turnos_extra_m[m,t] * model.X_M_adic[m]

  model.R17e = Constraint(model.m, model.t, rule = R17e_rule, doc = 'Eleccion de usar turnos adicionales')

  # R17a_c - Calculo de la disponibilidad de horas por corrugadora y por mes

  def R17a_c_rule(model, f, t):
    return model.DISP_HS_C[f,t] == model.Y_C_turnos_f[f]*model.horas_turno_fab

  model.R17a_c = Constraint(model.f, model.t, rule = R17a_c_rule, doc = 'Calculo de la disponibilidad de horas por corrugadora y por mes')

  # R17b_c - Calculo de la disponibilidad de horas extras por corrugadora y por mes

  def R17b_c_rule(model, f, t):
    return model.DISP_HS_EXTRA_C[f,t] == model.Y_C_turnos_extras[f]*model.horas_turno_fab

  model.R17b_c = Constraint(model.f, model.t, rule = R17b_c_rule, doc = 'Calculo de la disponibilidad de horas extras por corrugadora y por mes')

  # R17c - Cota maxima de turnos fijos y asociacion con la binaria

  def R17c_c_rule(model, f):
    return model.Y_C_turnos_f[f] <= model.turnos_fijos_min_c[f] - 1 + model.X_C_adic[f]

  model.R17c_c = Constraint(model.f, rule = R17c_c_rule, doc = 'Cota maxima de turnos fijos y asociacion con la binaria')

  # R17d - Cota minima de turnos fijos y asociacion con la binaria

  def R17d_c_rule(model, f):
    return model.turnos_fijos_min_c[f] - model.Y_C_turnos_f[f] <= model.M_turnos * ( 1 - model.X_C_adic[f] )

  model.R17d_c = Constraint(model.f, rule = R17d_c_rule, doc = 'Cota minima de turnos fijos y asociacion con la binaria')

  # R17e_c - Eleccion de usar turnos adicionales

  def R17e_c_rule(model, f, t):
    return model.Y_C_turnos_extras[f,t] <= model.turnos_extra_c[f,t] * model.X_C_adic[f]

  model.R17e_c = Constraint(model.f, model.t, rule = R17e_c_rule, doc = 'Eleccion de usar turnos adicionales')

  # RTurnos5_c - Disponibilidad horas fason corrugadoras

  def RTurnos5_c_rule(model, f, t):
    return model.SLACK_HS_C[f,t] == model.Y_C_turnos_slack[f,t] * model.horas_turno_fab

  model.RTurnos5_c = Constraint(model.f, model.t, rule = RTurnos5_c_rule, doc = 'Disponibilidad horas fason corrugadoras')

  # RTurnos5 - Disponibilidad horas fason maquinas

  def RTurnos5(model, m, t):
    return model.SLACK_HS_M[m,t] == model.Y_M_turnos_slack[m,t] * model.horas_turno_fab

  model.RTurnos5 = Constraint(model.m, model.t, rule = RTurnos5, doc = 'Disponibilidad horas fason maquinas')


################## turnos = 2 --> TURNOS POR BLOQUE #####################################################################################

if turnos == 2:

  # R_min_Corr

  def R_min_Corr_rule(model, f, t):
    return model.Y_C_turnos[f,t] >= model.turnos_c[f,t] * model.min_corr[f]/100

  model.R_min_Corr = Constraint(model.f, model.t, rule = R_min_Corr_rule, doc = 'R_min_Corr')

  # RTurnos1 - Maximo turnos disponibles maquinas

  def RTurnos1_rule(model, m, t):
    return model.Y_M_turnos[m,t] <= model.turnos_m[m,t]

  model.RTurnos1 = Constraint(model.m, model.t, rule = RTurnos1_rule, doc = 'Maximo turnos disponibles maquinas')

  # RTurnos2 - Maximo turnos extras disponibles maquinas

  def RTurnos2_rule(model, m, t):
    return model.Y_M_turnos_extras[m,t] <= model.turnos_extra_m[m,t]

  model.RTurnos2 = Constraint(model.m, model.t, rule = RTurnos2_rule, doc = 'Maximo turnos extras disponibles maquinas')

  # RTurnos3  - Disponibilidad horas normales maquinas

  def RTurnos3_rule(model, m, t):
    return model.DISP_HS_M[m,t] == model.Y_M_turnos[m,t] * model.horas_turno_fab

  model.RTurnos3 = Constraint(model.m, model.t, rule = RTurnos3_rule, doc = 'Disponibilidad horas normales maquinas')

  # RTurnos4 - Disponibilidad horas extras maquinas

  def Rturnos4_rule(model, m, t):
    return model.DISP_HS_EXTRA_M[m,t] == model.Y_M_turnos_extras[m,t] * model.horas_turno_fab

  model.Rturnos4 = Constraint(model.m, model.t, rule = Rturnos4_rule, doc = 'Disponibilidad horas extras maquinas')

  # RTurnos5 - Disponibilidad horas fason maquinas

  def Rturnos5_rule(model, m, t):
    return model.SLACK_HS_M[m,t] == model.Y_M_turnos_slack[m,t] * model.horas_turno_fab

  model.Rturnos5 = Constraint(model.m, model.t, rule = Rturnos5_rule, doc = 'Disponibilidad horas fason maquinas' )

  # RTurnos1_c - Maximo turnos disponibles corrugadora

  def RTurnos1_c_rule(model, f, t):
    return model.Y_C_turnos[f,t] <= model.turnos_c[f,t]

  model.RTurnos1_c = Constraint(model.f, model.t, rule = RTurnos1_c_rule, doc = 'Maximo turnos disponibles corrugadora')

  # RTurnos2_c - Maximo turnos extras disponibles corrugadora

  def RTurnos2_c_rule(model, f, t):
    return model.Y_C_turnos_extras[f,t] <= model.turnos_extra_c[f,t]

  model.RTurnos2_c = Constraint(model.f, model.t, rule = RTurnos2_c_rule, doc = 'Maximo turnos extras disponibles corrugadora')

  # RTurnos3_c  - Disponibilidad horas normales corrugadora

  def RTurnos3_c_rule(model, f, t):
    return model.DISP_HS_C[f,t] == model.Y_C_turnos[f,t] * model.horas_turno_fab

  model.RTurnos3_c = Constraint(model.f, model.t, rule = RTurnos3_c_rule, doc = 'Disponibilidad horas normales corrugadora')

  # RTurnos4_c - Disponibilidad horas extras corrugadora

  def Rturnos4_c_rule(model, f, t):
    return model.DISP_HS_EXTRA_C[f,t] == model.Y_C_turnos_extras[f,t] * model.horas_turno_fab

  model.Rturnos4_c = Constraint(model.f, model.t, rule = Rturnos4_c_rule, doc = 'Disponibilidad horas extras corrugadora')

  # RTurnos5_c - Disponibilidad horas fason corrugadora

  def Rturnos5_c_rule(model, f, t):
    return model.SLACK_HS_C[f,t] == model.Y_C_turnos_slack[f,t] * model.horas_turno_fab

  model.Rturnos5_c = Constraint(model.f, model.t, rule = Rturnos5_c_rule, doc = 'Disponibilidad horas fason corrugadora' )

  # Rturnos5_Bloques1 - Calculo del numero de bloques necesarios para cada grupo de tipo de maquinas

  def Rturnos5_Bloques1_rule(model, f, tm, t):
    if (f,tm) in model.F_TM:
      return sum( model.Y_M_turnos[m,t] for m in model.m if (f,tm,m) in model.F_TM_M ) + model.REMANENTE_NB[f,tm,t] == model.NB[f,tm,t] * model.dias_habiles[f,t]

  model.Rturnos5_Bloques1 = Constraint(model.f, model.tm, model.t, rule = Rturnos5_Bloques1, doc = 'Calculo del numero de bloques necesarios para cada grupo de tipo de maquinas')

  # Rturnos5_Bloques2 - Calculo del numero de bloques necesarios para corrugadoras

  def Rturnos5_Bloques2_rule(model, f, t):
    return model.Y_C_turnos[f,t] + model.REMANENTE_NB_C[f,t] == model.NB_C[f,t] * model.dias_habiles[f,t]

  model.Rturnos5_Bloques2 = Constraint(model.f, model.t, rule = Rturnos5_Bloques2_rule, doc = 'Calculo del numero de bloques necesarios para corrugadoras')

  # Rturnos5_Bloques3 - límite de bloques

  def Rturnos5_Bloques3_rule(model, f, tm, t):
    return model.NB[f,tm,t] <= 3*5

  model.Rturnos5_Bloques3 = Constraint(model.f, model.tm, model.t, rule = Rturnos5_Bloques3_rule, doc = 'limite de bloques')

  # Rturnos5_Bloques4 - límite de bloques por corrugadora

  def Rturnos5_Bloques4_rule(model, f, t):
    return model.NB_C[f,t] <= 3*5

  model.Rturnos5_Bloques4 = Constraint(model.f, model.t, rule = Rturnos5_Bloques4_rule, doc = 'límite de bloques por corrugadora')



## Ecuaciones Max - Min

In [ ]:
# ====  MIN / MAX DE CARTON POR FABRICA Y PERIODO ============================================

if min_max_carton == 0:

  # R11b - Minimo de carton por fabrica y periodo

  def R11b_rule(model, c, f, t):
    if (c,f) in model.CF:
      return model.reqmin_carton_t[c,f,t] * model.X_C[c,f,t] <= model.CANT_CORR_C[c,f,t] + model.slack_minC[c,f,t]

  model.R11b = Constraint(model.c, model.f, model.t, rule = R11b_rule, doc = 'Minimo de carton por fabrica y periodo')

  # R11c - Maximo de carton por fabrica y periodo

  def R11c_rule(model, c, f, t):
    if (c,f) in model.CF:
      return model.reqmax_carton_t[c,f,t] * model.X_C[c,f,t] >= model.CANT_CORR_C[c,f,t] - model.slack_maxC[c,f,t]

  model.R11c = Constraint(model.c, model.f, model.t, rule = R11c_rule, doc = 'Maximo de carton por fabrica y periodo')


# ====  MIN / MAX DE MP POR PROVEEDOR Y PERIODO ============================================

if min_max_abast == 1:

  # R10a - Calculo de la cantidad de toneladas a abastecer de materia prima mp desde el proveedor pr en el periodo t

  def R10a_rule(model, pr, t):
    return model.CANT_PR[pr,t] == sum( model.Q_A[mp,pr,f,t] for mp in model.mp for f in model.f if (pr, mp, f) in model.PRMPF)

  model.R10a = Constraint(model.pr, model.t, rule = R10a_rule, doc = 'Calculo de la cantidad de toneladas a abastecer de materia prima mp desde el proveedor pr en el periodo t')

  # R10b - Minimo a abastecer de materia prima mp desde el proveedor pr en el periodo t

  def R10b_rule(model, pr, t):
    return model.reqmin_pr[pr] * model.X_A_PR[pr,t] <= model.CANT_PR[pr,t]

  model.R10b = Constraint(model.pr, model.t, rule = R10b_rule, doc = 'Minimo a abastecer de materia prima mp desde el proveedor pr en el periodo t')

  # R10c - Maximo a abastecer de materia prima mp desde el proveedor pr en el periodo t

  def R10c_rule(model, pr, t):
    return model.reqmax_pr[pr] * model.X_A_PR[pr,t] >= model.CANT_PR[pr,t]

  model.R10c = Constraint(model.pr, model.t, rule = R10c_rule, doc = 'Maximo a abastecer de materia prima mp desde el proveedor pr en el periodo t')

if min_max_abast == 2:

  # R10d - Calculo de la cantidad de toneladas a abastecer de materia prima mp desde el proveedor pr en el periodo t

  def R10d_rule(model, pr, mp, t):
    if (pr,mp) in model.PRMP:
      return model.CANT_PRMP[pr,mp,t] == sum(model.Q_A[mp,pr,f,t] for f in model.f if (pr,mp,f) in model.PRMPF)

  model.R10d = Constraint(model.pr, model.mp, model.t, rule = R10d_rule, doc = 'Calculo de la cantidad de toneladas a abastecer de materia prima mp desde el proveedor pr en el periodo t')

  # R10e - Minimo a abastecer de materia prima mp desde el proveedor pr en el periodo t

  def R10e_rule(model, pr, mp, t):
    if (pr,mp) in model.PRMP:
      return model.reqmin_mppr[mp,pr] * model.X_A_PRMP[pr,mp,t] <= model.CANT_PRMP[pr,mp,t]

  model.R10e = Constraint(model.pr, model.mp, model.t, rule = R10e_rule, doc = 'Minimo a abastecer de materia prima mp desde el proveedor pr en el periodo t')

  # R10f - Maximo a abastecer de materia prima mp desde el proveedor pr en el periodo t

  def R10f_rule(model, pr, mp, t):
    if (pr,mp) in model.PRMP:
      return model.reqmax_mppr[mp,pr] * model.X_A_PRMP[pr,mp,t] >= model.CANT_PRMP[pr,mp,t]

  model.R10f = Constraint(model.pr, model.mp, model.t, rule = R10f_rule, doc = 'Maximo a abastecer de materia prima mp desde el proveedor pr en el periodo t')

if min_max_abast == 3:

  # R10g - Calculo de la cantidad de toneladas a abastecer de una familia de materia prima mp desde el proveedor pr en el periodo t

  def R10g_rule(model, pr, mp, t):
    if (pr, mp) in model.PRMP:
      return model.CANT_PRMP[pr,mp,t] == sum(model.Q_A[mp,pr,f,t] for f in model.f if (pr,mp,f) in model.PRMPF)

  model.R10g = Constraint(model.pr, model.mp, model.t, rule = R10g_rule, doc = 'Calculo de la cantidad de toneladas a abastecer de una familia de materia prima mp desde el proveedor pr en el periodo t')

  # R10h

  def R10h_rule(model, pr, fmp, t):
    if (fmp,pr) in model.FPR:
      return model.CANT_PRFAM[pr,fmp,t] == sum(model.CANT_PRMP[pr,mp,t] for mp in model.mp if (mp,fmp,pr) in model.MPFPR)

  model.R10h = Constraint(model.pr, model.fmp, model.t, rule = R10h_rule, doc = 'R10h')

  # R10i - Minimo a abastecer de una familia de materia prima mp desde el proveedor pr en el periodo t

  def R10i_rule(model, pr, fmp, t):
    if (fmp,pr) in model.FPR:
      return model.reqmin_prfam[pr,fmp] * model.X_A_FAM[pr,fmp,t] <= model.CANT_PRFAM[pr,fmp,t]

  model.R10i = Constraint(model.pr, model.fmp, model.t, rule = R10i_rule, doc = 'Minimo a abastecer de una familia de materia prima mp desde el proveedor pr en el periodo t')

  # R10j - Maximo a abastecer de una familia de materia prima mp desde el proveedor pr en el periodo t

  def R10j_rule(model, pr, fmp, t):
    if (fmp,pr) in model.FPR:
      return model.reqmax_prfam[pr,fmp] * model.X_A_FAM[pr,fmp,t] >= model.CANT_PRFAM[pr,fmp,t]

  model.R10j = Constraint(model.pr, model.fmp, model.t, rule = R10j_rule, doc = 'Maximo a abastecer de una familia de materia prima mp desde el proveedor pr en el periodo t')

if min_max_abast == 4:

  # R10k

  def R10k_rule(model, pr, f, t):
    return model.CANT_PRFAB[pr,f,t] == sum( model.Q_A[mp,pr,f,t] for mp in model.mp if (pr,mp,f) in model.PRMPF)

  model.R10k = Constraint(model.pr, model.f, model.t, rule = R10k_rule, doc = 'R10k')

  # R10l - Minimo a abastecer de un proveedro a una fábrica en el periodo t

  def R10l_rule(model, pr, f, t):
    return model.reqmin_prfab[pr,f] * model.X_A_F[pr,f,t] <= model.CANT_PRFAB[pr,f,t]

  model.R10l = Constraint(model.pr, model.f, model.t, rule = R10l_rule, doc = 'Minimo a abastecer de un proveedro a una fábrica en el periodo t')

  # R10m - Maximo a abastecer de un proveedro a una fábrica en el periodo t

  def R10m_rule(model, pr, f, t):
    return model.reqmax_prfab[pr,f] * model.X_A_F[pr,f,t] >= model.CANT_PRFAB[pr,f,t]

  model.R10m = Constraint(model.pr, model.f, model.t, rule = R10l_rule, doc = 'Maximo a abastecer de un proveedro a una fábrica en el periodo t')


# ====  MIN / MAX DE VIAJES DESDE CADA FABRICA A ZONA ============================================

if min_max_viajes == 1:

  # R12a - Calculo de la cantidad de viajes en el período t desde la fabrica f a la zona z

  def R12a_rule(model, f, zg, t):
    if (zg,f) in model.ZGF:
      return model.CANT_FZ[f,zg,t] == sum( ( model.Q_E[p,f,z,t] * 1000 ) / ( model.cant_cam[p,z,t] + 0.000001 ) for p in model.p for z in model.z if (zg,z) in model.ZGZ if (p,z) in model.PZ if (p,t) in model.PT )

  model.R12a = Constraint(model.f, model.zg, model.t, rule = R12a_rule, doc = 'Calculo de la cantidad de viajes en el período t desde la fabrica f a la zona z')

  # R12b - Minimo de viajes por período desde cada fábrica a zona

  def R12b_rule(model, f, zg, t):
    if (zg,f) in model.ZGF:
      return model.min_viajes[f,zg] * model.X_FZ[f,zg,t] <= model.CANT_FZ[f,zg,t]

  model.R12b = Constraint(model.f, model.zg, model.t, rule = R12b_rule, doc = 'Minimo de viajes por período desde cada fábrica a zona')

  # R12c - Maximo de viajes por período desde cada fábrica a zona

  def R12c_rule(model, f, zg, t):
    if (zg,f) in model.ZGF:
      return model.max_viajes[f,zg] * model.X_FZ[f,zg,t] >= model.CANT_FZ[f,zg,t]

  model.R12c = Constraint(model.f, model.zg, model.t, rule = R12c_rule, doc = 'Maximo de viajes por período desde cada fábrica a zona')



## Ecuaciones Abrochado - Pegado